# Imports

In [1]:
import pandas as pd
import gzip
import matplotlib.pyplot as plt
import seaborn as sns
import gc

DATA_DIR = 'data/'

# Utilitary functions

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

def clear_memory(df):
    lst = [df]
    del df
    del lst
    gc.collect()

# Number of reviews per category

In [4]:
def to_datetime(df):
    df['datetime'] = pd.to_datetime(df['unixReviewTime'], unit='s')
    
def get_count_year(df):
    return df['datetime'].groupby([df.datetime.dt.year]).agg('count')

def get_count_month_year(df):
    return df['datetime'].groupby([df.datetime.dt.year, df.datetime.dt.month]).agg('count')

## Category: Pet Supplies

In [ ]:
pet_supplies = getDF(DATA_DIR + 'reviews_Pet_Supplies.json.gz')
pet_supplies.head()

In [ ]:
pet_supplies_year = get_count_year(pet_supplies)
pet_supplies_month_year = get_count_month_year(pet_supplies)
clear_memory(pet_supplies)

## Category: Office Products

In [ ]:
office = getDF(DATA_DIR + 'reviews_Office_Products.json.gz')
office = to_datetime(office)
office.head()

In [ ]:
office_year = get_count_year(office)
office_month_year = get_count_month_year(office)
clear_memory(office)

## Category: Grocery and Gourmet Food

In [3]:
food = getDF(DATA_DIR + 'reviews_Grocery_and_Gourmet_Food.json.gz')
food = to_datetime(food)
food.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1ZQZ8RJS1XVTX,0657745316,gsxrgirl,"[0, 0]","No sugar, no GMO garbage, no fillers that come...",5.0,Best vanilla I've ever had,1381449600,"10 11, 2013"
1,A31W38VGZAUUM4,0700026444,FIFA Lvr,"[1, 1]","This is my absolute, undisputed favorite tea r...",5.0,Terrific Tea!,1354752000,"12 6, 2012"
2,A3I0AV0UJX5OH0,1403796890,Alicia b,"[0, 0]",I ordered spongbob slippers and I got John Cen...,1.0,grrrrrrr,1385942400,"12 2, 2013"
3,A3QAAOLIXKV383,1403796890,"Danny K. Tilley ""Dan Tilley""","[0, 0]",The cart is fine and works for the purpose for...,3.0,Storage on Wheels Cart,1307836800,"06 12, 2011"
4,AB1A5EGHHVA9M,141278509X,CHelmic,"[1, 1]",This product by Archer Farms is the best drink...,5.0,The best drink mix,1332547200,"03 24, 2012"


In [ ]:
food_year = get_count_year(food)
food_month_year = get_count_month_year(food)
clear_memory(food)